## Projeto

In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from pennylane.optimize import AdamOptimizer
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.embeddings import AngleEmbedding
from tqdm.notebook import tqdm, trange

In [2]:

# Read the HDF5 file using pandas
data_frame_fcnc = pd.read_hdf('fcnc_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_fcnc.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_fcnc.head()


Number of rows: 90548


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,1.408853,15.150869,339.182312,2.350262,1.396943,0.710451,0.109013,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,test,7.762202e-09,0.001285
1,1,-2.481838,7.208333,247.036240,-2.280740,0.428710,0.205213,0.000000,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,val,7.762202e-09,0.001285
2,0,1.476267,94.220718,238.014694,-1.788097,94.256210,2.418446,1.585315,1.127324,0.431098,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
3,1,0.647480,13.459283,230.971832,-1.032663,1.227122,0.467150,0.164008,0.000000,0.000000,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285
4,0,2.106436,97.490242,698.399902,-3.059983,36.555862,2.937936,1.799140,1.093004,0.589724,...,0,None,PyDelphes,signal,1,tZFCNC,tZFCNC_PyDelphes,train,7.762249e-09,0.001285


In [3]:
# Read the HDF5 file using pandas
data_frame_bkg = pd.read_hdf('bkg_pythia_sanitised_features.h5')

# Get the number of rows
num_rows = data_frame_bkg.shape[0]

print('Number of rows: {}'.format(num_rows))

# Explore the data
data_frame_bkg.head()


Number of rows: 1002490


,Electron_Multi,FatJet1_Eta,FatJet1_Mass,FatJet1_PT,FatJet1_Phi,FatJet1_Tau1,FatJet1_Tau2,FatJet1_Tau3,FatJet1_Tau4,FatJet1_Tau5,...,gen_decay2,gen_decay_filter,gen_filter,gen_label,gen_n_btags,gen_sample,gen_sample_filter,gen_split,gen_weights,gen_xsec
0,2,-1.988600,52.710262,229.350952,0.728242,36.148926,23.039709,16.949991,14.424411,12.000529,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,train,0.000018,11.9635
1,0,0.528382,61.115589,315.538910,-0.863614,32.592808,22.366640,16.285843,13.938633,11.180016,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,test,0.000018,11.9635
2,0,1.479911,98.012802,251.109573,-3.133624,90.252274,33.646885,30.612156,27.973904,23.729696,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
3,2,0.926899,-0.000007,240.909348,0.835656,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635
4,0,0.781194,72.234299,206.020386,-0.320449,48.886372,20.743645,16.572512,13.070706,11.269534,...,0,2L,HT250to500,bkg,1,Zjj,Zjj_HT250to500,val,0.000018,11.9635


In [4]:
# Z-score normalization

# copy the data to a new dataframe
data_frame_fcnc_norm = data_frame_fcnc.copy()
data_frame_bkg_norm = data_frame_bkg.copy()

# normalize the data except the categorical features and the weights
for feature in data_frame_fcnc.columns:
    if feature in ['gen_decay_filter', 'gen_filter', 'gen_label', 'gen_n_btags', 'gen_sample', 'gen_sample_filter', 'gen_split', 'gen_decay2','gen_decay1', 'gen_xsec']:
        pass
    else: 
        data_frame_fcnc_norm[feature] = (data_frame_fcnc[feature] - data_frame_fcnc[feature].mean()) / data_frame_fcnc[feature].std()
        
for feature in data_frame_bkg.columns:
    if feature in ['gen_decay_filter', 'gen_filter', 'gen_label', 'gen_n_btags', 'gen_sample', 'gen_sample_filter', 'gen_split','gen_decay2','gen_decay1','gen_xsec']:
        pass
    else:  
        data_frame_bkg_norm[feature] = (data_frame_bkg[feature] - data_frame_bkg[feature].mean()) / data_frame_bkg[feature].std()

### VQC

#### Pennylane (V1)

In [5]:
# Split the original data into train, validation and test sets for each dataset
data_frame_fcnc_train = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'train']
data_frame_bkg_train =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'train']

data_frame_fcnc_test = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'test']
data_frame_bkg_test =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'test']

data_frame_fcnc_val = data_frame_fcnc.loc[data_frame_fcnc['gen_split'] == 'val']
data_frame_bkg_val =  data_frame_bkg.loc[data_frame_bkg['gen_split'] == 'val']


# get 500 points of each dataset and  join the datasets (randomly)
train_fcnc = data_frame_fcnc_train.sample(n=500, random_state=42)
train_bkg = data_frame_bkg_train.sample(n=500, random_state=42)
train = pd.concat([train_fcnc, train_bkg])
train = train.sample(frac=1, random_state=42)

test_fcnc = data_frame_fcnc_test.sample(n=500, random_state=42)
test_bkg = data_frame_bkg_test.sample(n=500, random_state=42)
test= pd.concat([test_fcnc, test_bkg])
test = test.sample(frac=1,random_state=42)

val_fnc = data_frame_fcnc_val.sample(n=500, random_state=42)
val_bkg = data_frame_bkg_val.sample(n=500, random_state=42)
val = pd.concat([val_fnc, val_bkg])
val = val.sample(frac=1, random_state=42)

# get the weights for each dataset
w_train = train[['gen_xsec']]
w_test = test[['gen_xsec']]
w_val = val[['gen_xsec']]


# change the signal and bkg labels to 0 and 1 and get the labels for each dataset
train = train.replace(['signal'], 1)
train= train.replace(['bkg'], 0)
y_train = train[['gen_label']]
x_train = train[['MissingET_MET', 'Jet1_BTag']]

test = test.replace(['signal'], 1)
test= test.replace(['bkg'], 0)
y_test = test[['gen_label']]
x_test = test[['MissingET_MET', 'Jet1_BTag']]

val = val.replace(['signal'], 1)
val= val.replace(['bkg'], 0)
y_val = val[['gen_label']]
x_val = val[['MissingET_MET', 'Jet1_BTag']]

y_train_arr = np.concatenate(y_train.values, axis=0 )
y_val_arr = np.concatenate( y_val.values, axis=0 )
y_test_arr = np.concatenate( y_test.values, axis=0 )

# Renormalize data weights
w_train[y_train_arr == 1] = (w_train[y_train_arr == 1] / w_train[y_train_arr == 1].sum()) * y_train_arr.shape[0] / 2
w_train[y_train_arr == 0] = (w_train[y_train_arr == 0] / w_train[y_train_arr == 0].sum()) * y_train_arr.shape[0] / 2
        
w_test[y_test_arr == 1] = (w_test[y_test_arr == 1] / w_test[y_test_arr == 1].sum()) * w_test.shape[0] / 2
w_test[y_test_arr == 0] = (w_test[y_test_arr == 0] / w_test[y_test_arr == 0].sum()) * w_test.shape[0] / 2
        
w_val[y_test_arr == 1] = (w_val[y_test_arr == 1] / w_val[y_test_arr == 1].sum()) * w_val.shape[0] / 2
w_val[y_test_arr == 0] = (w_val[y_test_arr == 0] / w_val[y_test_arr == 0].sum()) * w_val.shape[0] / 2

# Concatenate features
X = np.concatenate([x_train, x_test,x_val])

# Normalize the data for angle embeding  (Put the data between -pi and pi)
X = (((X - X.min()) / (X.max() - X.min())) * 2 - 1) * (np.pi)

# Split the features array into train, validation and test sets
x_train = X[:1000]
x_test = X[1000:2000]
x_val = X[2000:]

In [6]:
# accuracy function
def accuracy(labels, predictions):

    loss = 0
    for l, p in zip(labels, predictions):
        if abs(l - p) < 1e-5:
            loss = loss + 1
    loss = loss / len(labels)

    return loss


In [7]:
# loss function
def square_loss(labels, predictions):
    loss = 0
    for l, p in zip(labels, predictions):
        loss = loss + (l - p) ** 2

    loss = loss / len(labels)
    return loss

# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

# classifier function    
def classifier(n_features, n_layers, weights, x):
        #c = circuit(n_features, n_layers, weights, x)
        dev=qml.device("default.qubit", wires=n_features)
        return qml.QNode(circuit, dev)(n_features, n_layers, weights, x)
    
# cost function    
def cost(n_features, n_layers,weights,X,Y,W):  
        # Compute predictions
        y_scores = [(classifier(n_features, n_layers,weights, x) + 1) / 2 for x in X]

        loss = square_loss(Y, y_scores)
        loss = loss * W
        loss = loss.sum()
        
        return loss
    
# train step function    
def train_step(n_features, n_layers,x_train,y_train, w_train, weights, opt,desc='Training'):
        
        # Only require grad if necessary
        x_train = np.array(x_train, requires_grad=False)
        y_train = np.array(y_train, requires_grad=True)
        w_train = np.array(w_train, requires_grad=False)

        # Compute cost and update weights
        weights, loss = opt.step_and_cost(cost, n_features, n_layers,weights, X=x_train, Y=y_train, W=w_train)

        return loss, weights
    
# validation step function
def validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch,best_weights,desc='Validation'):
    X_val = np.array(x_val, requires_grad=False)
    Y_val = np.array(y_val, requires_grad=False)
    W_val = np.array(w_val, requires_grad=False)

    y_scores = np.array([classifier(n_features, n_layers, weights, x) for x in X_val])
    y_scores = (y_scores + 1) / 2

    W_val[Y_val == 1] = (W_val[Y_val == 1] / W_val[Y_val == 1].sum()) * W_val.shape[0] / 2
    W_val[Y_val == 0] = (W_val[Y_val == 0] / W_val[Y_val == 0].sum()) * W_val.shape[0] / 2

    auc_score = roc_auc_score(y_true=Y_val, y_score=y_scores, sample_weight=W_val)
    loss = cost(n_features, n_layers, weights, X_val, Y_val, W_val)


    if best_score is None or auc_score > best_score:
        best_score = auc_score
        best_score_epoch = epoch_number
        best_weights = weights

    tqdm.write(f"Epoch: {epoch_number}, Validation Loss: {loss:.4f}, AUC Score: {auc_score:.4f}")

    return best_score, best_score_epoch, best_weights
        
        
# train function
def train(n_features, n_layers, x_train, y_train, learning_rate, weights, max_epochs, epoch_number):
    opt = AdamOptimizer(learning_rate)
    best_score = None
    best_weights = None
    best_score_epoch = None

    with tqdm(total=max_epochs, desc='Epoch', unit='epoch') as pbar:
        for epoch in range(epoch_number, max_epochs):
            epoch_number = epoch

            loss, nf_nl_weights = train_step(n_features, n_layers, x_train, y_train, w_train, weights, opt, desc='Training')
            
            # Log variable values using tqdm.write
            tqdm.write(f"Epoch: {epoch_number:}, Loss: {loss:.4f}")
            
            
            weights = nf_nl_weights[2:]
            weights = weights[0]

            if epoch_number == max_epochs - 1 or (epoch_number+1)%5==0:
                best_score, best_score_epoch, best_weights = validation_step(n_features, n_layers, x_val, y_val, w_val, weights, best_score, epoch_number, best_score_epoch, best_weights,desc='Validation')
                # early stopping
                if epoch_number - best_score_epoch > 30 and epoch_number > 80:
                    tqdm.write(f"Early stopping at epoch {epoch_number}")
                    break

            pbar.update(1)  # Update progress bar
        tqdm.write(f"Best Score: {best_score:.4f}")            
        
    return best_score, best_weights

def test(n_features, n_layers,x_test,y_test,w_test, weights):
        # Remove grad
        X_test = np.array(x_test, requires_grad=False)
        Y_test = np.array(y_test, requires_grad=False)
        W_test = np.array(w_test, requires_grad=False)

        # This will be between -1 and 1, we need to convert to between 0 and 1
        y_scores = np.array([classifier(n_features, n_layers,weights, x) for x in X_test])
        y_scores = (y_scores + 1) / 2

        # Renormalize weights
        W_test[Y_test == 1] = (W_test[Y_test == 1] / W_test[Y_test == 1].sum()) * W_test.shape[0] / 2
        W_test[Y_test == 0] = (W_test[Y_test == 0] / W_test[Y_test == 0].sum()) * W_test.shape[0] / 2

        # Calculate ROC
        auc_score = roc_auc_score(y_true=Y_test, y_score=y_scores, sample_weight=W_test)
        
        return auc_score
    

### Original

In [8]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)


Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 507.6642
Epoch: 1, Loss: 507.0090
Epoch: 2, Loss: 505.8304
Epoch: 3, Loss: 504.0940
Epoch: 4, Loss: 501.7893
Epoch: 4, Validation Loss: 493.7559, AUC Score: 0.2792
Epoch: 5, Loss: 498.9076
Epoch: 6, Loss: 495.4493
Epoch: 7, Loss: 491.4242
Epoch: 8, Loss: 486.8508
Epoch: 9, Loss: 481.7572
Epoch: 9, Validation Loss: 471.3341, AUC Score: 0.2791
Epoch: 10, Loss: 476.1815
Epoch: 11, Loss: 470.1706
Epoch: 12, Loss: 463.7779
Epoch: 13, Loss: 457.0577
Epoch: 14, Loss: 450.0574
Epoch: 14, Validation Loss: 438.5077, AUC Score: 0.2794
Epoch: 15, Loss: 442.8126
Epoch: 16, Loss: 435.3477
Epoch: 17, Loss: 427.6760
Epoch: 18, Loss: 419.8019
Epoch: 19, Loss: 411.7330
Epoch: 19, Validation Loss: 399.9954, AUC Score: 0.2798
Epoch: 20, Loss: 403.4872
Epoch: 21, Loss: 395.0909
Epoch: 22, Loss: 386.5757
Epoch: 23, Loss: 377.9750
Epoch: 24, Loss: 369.3239
Epoch: 24, Validation Loss: 358.1327, AUC Score: 0.2806
Epoch: 25, Loss: 360.6597
Epoch: 26, Loss: 352.0220
Epoch: 27, Loss: 343.4526
Epoc

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7339739775470611


#### Type of rotation in  angle embeding

In [ ]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 258.5485
Epoch: 1, Loss: 257.6502
Epoch: 2, Loss: 256.5583
Epoch: 3, Loss: 255.2702
Epoch: 4, Loss: 253.7911
Epoch: 5, Loss: 252.1263
Epoch: 6, Loss: 250.2784
Epoch: 7, Loss: 248.2543
Epoch: 8, Loss: 246.0634
Epoch: 9, Loss: 243.7140
Epoch: 9, Validation Loss: 248.7585, AUC Score: 0.1085
Epoch: 10, Loss: 241.2155
Epoch: 11, Loss: 238.5806
Epoch: 12, Loss: 235.8232
Epoch: 13, Loss: 232.9568
Epoch: 14, Loss: 229.9980
Epoch: 15, Loss: 226.9645
Epoch: 16, Loss: 223.8732
Epoch: 17, Loss: 220.7421
Epoch: 18, Loss: 217.5906
Epoch: 19, Loss: 214.4355
Epoch: 19, Validation Loss: 219.3143, AUC Score: 0.1096
Epoch: 20, Loss: 211.2922
Epoch: 21, Loss: 208.1762
Epoch: 22, Loss: 205.1005
Epoch: 23, Loss: 202.0744
Epoch: 24, Loss: 199.1045
Epoch: 25, Loss: 196.1948
Epoch: 26, Loss: 193.3455
Epoch: 27, Loss: 190.5537
Epoch: 28, Loss: 187.8137
Epoch: 29, Loss: 185.1173
Epoch: 29, Validation Loss: 189.1727, AUC Score: 0.1121
Epoch: 30, Loss: 182.4546
Epoch: 31, Loss: 179.8147
Epoch: 32, 

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7353790344559314


#### Retirar rotações

In [ ]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            #W1 = weights[layer]

            # Define Rotations
            #for i in range(0,n_features):
                #qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 0, Loss: 257.8696
Epoch: 1, Loss: 257.8696
Epoch: 2, Loss: 257.8696
Epoch: 3, Loss: 257.8696
Epoch: 4, Loss: 257.8696
Epoch: 5, Loss: 257.8696
Epoch: 6, Loss: 257.8696
Epoch: 7, Loss: 257.8696
Epoch: 8, Loss: 257.8696
Epoch: 9, Loss: 257.8696
Epoch: 9, Validation Loss: 264.0529, AUC Score: 0.1085
Epoch: 10, Loss: 257.8696
Epoch: 11, Loss: 257.8696
Epoch: 12, Loss: 257.8696
Epoch: 13, Loss: 257.8696
Epoch: 14, Loss: 257.8696
Epoch: 15, Loss: 257.8696


/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 16, Loss: 257.8696
Epoch: 17, Loss: 257.8696
Epoch: 18, Loss: 257.8696
Epoch: 19, Loss: 257.8696
Epoch: 19, Validation Loss: 264.0529, AUC Score: 0.1085


...

/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 50, Loss: 257.8696
Epoch: 51, Loss: 257.8696
Epoch: 52, Loss: 257.8696
Epoch: 53, Loss: 257.8696
Epoch: 54, Loss: 257.8696
Epoch: 55, Loss: 257.8696
Epoch: 56, Loss: 257.8696
Epoch: 57, Loss: 257.8696
Epoch: 58, Loss: 257.8696
Epoch: 59, Loss: 257.8696
Epoch: 59, Validation Loss: 264.0529, AUC Score: 0.1085
Epoch: 60, Loss: 257.8696
Epoch: 61, Loss: 257.8696
Epoch: 62, Loss: 257.8696
Epoch: 63, Loss: 257.8696
Epoch: 64, Loss: 257.8696
Epoch: 65, Loss: 257.8696
Epoch: 66, Loss: 257.8696
Epoch: 67, Loss: 257.8696
Epoch: 68, Loss: 257.8696
Epoch: 69, Loss: 257.8696
Epoch: 69, Validation Loss: 264.0529, AUC Score: 0.1085
Epoch: 70, Loss: 257.8696
Epoch: 71, Loss: 257.8696
Epoch: 72, Loss: 257.8696


/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")
/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


Epoch: 73, Loss: 257.8696
Epoch: 74, Loss: 257.8696
Epoch: 75, Loss: 257.8696
Epoch: 76, Loss: 257.8696
Epoch: 77, Loss: 257.8696
Epoch: 78, Loss: 257.8696
Epoch: 79, Loss: 257.8696
Epoch: 79, Validation Loss: 264.0529, AUC Score: 0.1085
Epoch: 80, Loss: 257.8696
Epoch: 81, Loss: 257.8696
Epoch: 82, Loss: 257.8696
Epoch: 83, Loss: 257.8696
Epoch: 84, Loss: 257.8696
Epoch: 85, Loss: 257.8696
Epoch: 86, Loss: 257.8696
Epoch: 87, Loss: 257.8696
Epoch: 88, Loss: 257.8696
Epoch: 89, Loss: 257.8696
Epoch: 89, Validation Loss: 264.0529, AUC Score: 0.1085
Epoch: 90, Loss: 257.8696
Epoch: 91, Loss: 257.8696
Epoch: 92, Loss: 257.8696
Epoch: 93, Loss: 257.8696
Epoch: 94, Loss: 257.8696
Epoch: 95, Loss: 257.8696
Epoch: 96, Loss: 257.8696
Epoch: 97, Loss: 257.8696
Epoch: 98, Loss: 257.8696
Epoch: 99, Loss: 257.8696
Epoch: 99, Validation Loss: 264.0529, AUC Score: 0.1085
Best Score: 0.1085


/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")
/home/brunasalgado/LIP/LIP_QML_2023/.envLip/lib/python3.9/site-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")


In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.25819930973022687


### Retirar entanglement

In [ ]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)


        return qml.expval(qml.PauliZ(0))

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 166.4724
Epoch: 1, Loss: 163.5699
Epoch: 2, Loss: 160.6324
Epoch: 3, Loss: 157.6568
Epoch: 4, Loss: 154.6489
Epoch: 5, Loss: 151.6227
Epoch: 6, Loss: 148.5940
Epoch: 7, Loss: 145.5718
Epoch: 8, Loss: 142.5558
Epoch: 9, Loss: 139.5553
Epoch: 9, Validation Loss: 128.1398, AUC Score: 0.8914
Epoch: 10, Loss: 136.5854
Epoch: 11, Loss: 133.6609
Epoch: 12, Loss: 130.7916
Epoch: 13, Loss: 127.9825
Epoch: 14, Loss: 125.2430
Epoch: 15, Loss: 122.5864
Epoch: 16, Loss: 120.0259
Epoch: 17, Loss: 117.5706
Epoch: 18, Loss: 115.2263
Epoch: 19, Loss: 113.0003
Epoch: 19, Validation Loss: 102.5228, AUC Score: 0.8902
Epoch: 20, Loss: 110.9015
Epoch: 21, Loss: 108.9384
Epoch: 22, Loss: 107.1162
Epoch: 23, Loss: 105.4373
Epoch: 24, Loss: 103.9034
Epoch: 25, Loss: 102.5161
Epoch: 26, Loss: 101.2762
Epoch: 27, Loss: 100.1818
Epoch: 28, Loss: 99.2288
Epoch: 29, Loss: 98.4116
Epoch: 29, Validation Loss: 89.9679, AUC Score: 0.8878
Epoch: 30, Loss: 97.7234
Epoch: 31, Loss: 97.1567
Epoch: 32, Loss:

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7415599779037287


#### Substituir CNOTs por CZ

In [ ]:
# quantum circuit function
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="X" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CZ(wires=[i, 0])
                        else:
                            qml.CZ(wires=[i, i + 1])
                else:
                    qml.CZ(wires=[1, 0])

        return qml.expval(qml.PauliZ(0))

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 166.1363
Epoch: 1, Loss: 166.0886
Epoch: 2, Loss: 165.9619
Epoch: 3, Loss: 165.6897
Epoch: 4, Loss: 165.2375
Epoch: 5, Loss: 164.5753
Epoch: 6, Loss: 163.6866
Epoch: 7, Loss: 162.5604
Epoch: 8, Loss: 161.1900
Epoch: 9, Loss: 159.5728
Epoch: 9, Validation Loss: 150.1490, AUC Score: 0.8915
Epoch: 10, Loss: 157.7106
Epoch: 11, Loss: 155.6095
Epoch: 12, Loss: 153.2802
Epoch: 13, Loss: 150.7378
Epoch: 14, Loss: 148.0018
Epoch: 15, Loss: 145.0960
Epoch: 16, Loss: 142.0485
Epoch: 17, Loss: 138.8908
Epoch: 18, Loss: 135.6577
Epoch: 19, Loss: 132.3857
Epoch: 19, Validation Loss: 121.7375, AUC Score: 0.8914
Epoch: 20, Loss: 129.1130
Epoch: 21, Loss: 125.8779
Epoch: 22, Loss: 122.7176
Epoch: 23, Loss: 119.6670
Epoch: 24, Loss: 116.7577
Epoch: 25, Loss: 114.0168
Epoch: 26, Loss: 111.4663
Epoch: 27, Loss: 109.1221
Epoch: 28, Loss: 106.9946
Epoch: 29, Loss: 105.0886
Epoch: 29, Validation Loss: 96.2832, AUC Score: 0.8886
Epoch: 30, Loss: 103.4038
Epoch: 31, Loss: 101.9354
Epoch: 32, L

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7417816820371923


#### Medir o segundo qubit

In [ ]:
def circuit(n_features, n_layers, weights, x):
        # Embedding
        
        qml.AngleEmbedding(x,range (0, n_features),rotation="Y" )

        # For every layer
        for layer in range(n_layers):
            W1 = weights[layer]

            # Define Rotations
            for i in range(0,n_features):
                qml.Rot(W1[i, 0], W1[i, 1], W1[i, 2], wires=i)

            # Entanglement
            if n_features != 1:
                if n_features > 2:
                    for i in range(n_features):
                        if i == n_features - 1:
                            qml.CNOT(wires=[i, 0])
                        else:
                            qml.CNOT(wires=[i, i + 1])
                else:
                    qml.CNOT(wires=[1, 0])

        return qml.expval(qml.PauliZ(1))

In [ ]:
# number of features and layers
n_features = 2
n_layers = 3

# Random weight initialization
weights = 0.01 * np.random.randn(n_layers, n_features, 3, requires_grad=True)

# We create a quantum device with n_features "wires" (or qubits)
dev = qml.device('default.qubit', wires=n_features)

# train the model
best_score, best_weights = train (n_features, n_layers,x_train,y_train_arr,0.01, weights, 100, 0)

Epoch:   0%|          | 0/100 [00:00<?, ?epoch/s]

Epoch: 0, Loss: 249.9795
Epoch: 1, Loss: 249.8142
Epoch: 2, Loss: 249.5087
Epoch: 3, Loss: 249.0463
Epoch: 4, Loss: 248.4191
Epoch: 5, Loss: 247.6203
Epoch: 6, Loss: 246.6436
Epoch: 7, Loss: 245.4829
Epoch: 8, Loss: 244.1333
Epoch: 9, Loss: 242.5903
Epoch: 9, Validation Loss: 240.7502, AUC Score: 0.8550
Epoch: 10, Loss: 240.8503
Epoch: 11, Loss: 238.9108
Epoch: 12, Loss: 236.7710
Epoch: 13, Loss: 234.4305
Epoch: 14, Loss: 231.8906
Epoch: 15, Loss: 229.1553
Epoch: 16, Loss: 226.2289
Epoch: 17, Loss: 223.1177
Epoch: 18, Loss: 219.8311
Epoch: 19, Loss: 216.3793
Epoch: 19, Validation Loss: 212.5456, AUC Score: 0.8295
Epoch: 20, Loss: 212.7745
Epoch: 21, Loss: 209.0317
Epoch: 22, Loss: 205.1672
Epoch: 23, Loss: 201.1988
Epoch: 24, Loss: 197.1458
Epoch: 25, Loss: 193.0287
Epoch: 26, Loss: 188.8676
Epoch: 27, Loss: 184.6834
Epoch: 28, Loss: 180.4975
Epoch: 29, Loss: 176.3319
Epoch: 29, Validation Loss: 171.9730, AUC Score: 0.7974
Epoch: 30, Loss: 172.2096
Epoch: 31, Loss: 168.1539
Epoch: 32, 

In [ ]:
# test the model
auc_score = test(n_features, n_layers,x_test,y_test,w_test, best_weights)
print (auc_score)

0.7302230317327565
